In [1]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast
import attributes


from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

/home/jovyan/attributes.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['Object'] = obj


In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"
AIRFLOW_WASSTRAAT_CONFIG_NEW = "./wasstraat_config/Wasstraat_Config_Harmonize_new.xlsx"

SUGGESTIE_XL = "./wasstraat_config/Wasstraat_Attribuut_Suggestie_v2.xlsx"

In [3]:
df_attributes = attributes.getAllAttributes()
df_attributes[df_attributes.Object == 'Artefact'].head(50)

,Object,Kolommen,omschrijvingen,count,projecten,Attribute,teller,percentage_gevuld
140,Artefact,(none),[],3,"[DC179, DC24_STADSKANTOOR, DC154]",,NaN,NaN
141,Artefact,ARTEFACT,[administratief nummer],7,"[DC179, DC008, DC24_STADSKANTOOR, DC154]",artefactnr,203.0,100.0
142,Artefact,BEGINDAT,[],2,"[DC179, DC24_STADSKANTOOR]",dateringvanaf,NaN,NaN
143,Artefact,BESCHRYF,[],2,"[DC179, DC24_STADSKANTOOR]",beschrijving,NaN,NaN
144,Artefact,COMPLEET,[],2,"[DC179, DC24_STADSKANTOOR]",compleetheid,NaN,NaN
145,Artefact,DOOSNO,[],2,"[DC179, DC24_STADSKANTOOR]",doosnr,NaN,NaN
146,Artefact,EINDDAT,[],2,"[DC179, DC24_STADSKANTOOR]",dateringtot,NaN,NaN
147,Artefact,FOTO,[],2,"[DC179, DC24_STADSKANTOOR]",fotonr,NaN,NaN
148,Artefact,FOTOGRAAF,[],2,"[DC179, DC24_STADSKANTOOR]",,NaN,NaN
149,Artefact,FUNCTIE,[],2,"[DC179, DC24_STADSKANTOOR]",functievoorwerp,NaN,NaN


In [10]:
lst_artefact_attributes = attributes.getArtefactAttributes()
lst_artefact_attributes



['ABRcodering',
 'aantal',
 'afmetingen',
 'artefactnr',
 'beschrijving',
 'bibliografie',
 'catalogus',
 'compleetheid',
 'conservering',
 'datering',
 'dateringtot',
 'dateringvanaf',
 'diversen',
 'doosnr',
 'exposabel',
 'fotonr',
 'functievoorwerp',
 'gewicht',
 'groep',
 'literatuur',
 'mai',
 'maten',
 'materiaalcode',
 'materiaalspecifiek',
 'naam_voorwerp',
 'opmerkingen',
 'origine',
 'plek',
 'publicatiecode',
 'putnr',
 'restauratie',
 'soortvoorwerp',
 'spoornr',
 'subnr',
 'tekeningnr',
 'typenaam',
 'typevoorwerp',
 'versiering',
 'vlaknr',
 'vondstnr',
 'vondstomstandigheden',
 'weggegooid']

In [25]:
df_group = df_attributes.groupby(['Object', 'Attribute']).agg({'omschrijvingen':sum}).reset_index()
df_group['omschrijving'] = df_group.apply(lambda x: max(x['omschrijvingen'], key=len) if len(x['omschrijvingen']) > 0 else "", axis=1)
#df_group[(df_group.Object == 'Aardewerk') & (~df_group.Attribute.isin(lst_artefact_attributes))]
df_group

,Object,Attribute,omschrijvingen,omschrijving
0,Aardewerk,,"[KEUZELIJST AARDEWERK 1 BAKSEL, KEUZELIJST AAR...",KEUZELIJST AARDEWERK 2 VOORWERP
1,Aardewerk,aantal,[aantal van dezelfde scherven per vondstnummer...,aantal van dezelfde scherven per vondstnummer
2,Aardewerk,artefactnr,"[artefact nummer, geeft invoervolgorde automat...",geeft invoervolgorde automatisch aan
3,Aardewerk,baksel,"[soort AW, baksel, soort AW, baksel]",soort AW
4,Aardewerk,bakseltype,[bakseltype],bakseltype
...,...,...,...,...
558,Vulling,spoornr,"[spoornummer, spoornummer]",spoornummer
559,Vulling,sublaag,[sublaag],sublaag
560,Vulling,textuur,"[textuur van de bodem, textuur]",textuur van de bodem
561,Vulling,textuurbijmenging,[textuur bijmenging],textuur bijmenging


In [26]:
lst_aanw = []
df_artf = df_group[(df_group.Object == 'Aardewerk') & (~df_group.Attribute.isin(lst_artefact_attributes)) & (df_group.Attribute != "")]
for index, row in df_artf.iterrows():
    if not row['Attribute'] in lst_aanw:
        print(f'{row.Attribute} = Column(String(200), comment="{row.omschrijving}")')
    



baksel = Column(String(200), comment="soort AW")
bakseltype = Column(String(200), comment="bakseltype")
bodem = Column(String(200), comment="vorm van de bodem/voet")
categorie = Column(String(200), comment="categorie")
codes = Column(String(200), comment="codes van type")
decoratie = Column(String(200), comment="decoratie")
diameter = Column(String(200), comment="")
fragment = Column(String(200), comment="fragment: rand, wand, bodem, oor etc")
glazuur = Column(String(200), comment="glazuur")
grootste_diameter = Column(String(200), comment="grootste diameter in millimeters, 0 = niet gemeten")
hoogte = Column(String(200), comment="hoogte in millimeters, 0 = niet gemeten")
kleur = Column(String(200), comment="kleur")
materiaal = Column(String(200), comment="materiaal")
materiaalsoort = Column(String(200), comment="materiaalsoort")
max_diameter = Column(String(200), comment="")
oor = Column(String(200), comment="vorm en versiering van oor")
past_aan = Column(String(200), comment="vondstnum